In [1]:
import os
import random
from Mydataset import MYDataset
import math
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import r2_score
from IPython.display import clear_output as clear
import statsmodels.api as sm
from SWNN import GNNWR

In [2]:
varName = ['fCO2', 'Chl', 'Temp', 'Salt']

dataset = pd.read_csv("D://CO2_data3.csv", encoding="utf-8")
dataset = dataset.dropna()
dataset = dataset[dataset.index % 4 == 0]

df0 = dataset['date'].str.split("/",expand = True)
df0.columns = ['year', 'month', 'date']

dataset['month'] = df0['month']
dataset = dataset[dataset.month == '7']

train_li = random.sample([i for i in range(0, dataset.shape[0])], int(0.8 * dataset.shape[0]))
train_li.sort()

j = 0
test_li = []

for i in range(0, dataset.shape[0], 1):
    if i != train_li[j] | j >= len(train_li):
        test_li.append(i)
    else:
        j = j + 1

train_set = dataset.iloc[train_li, :]
test_set  = dataset.iloc[test_li,  :]

mean_li = []
std_li = []

for i in range(0, len(varName), 1):
    mean_li.append(train_set[varName[i]].mean())
    std_li.append(train_set[varName[i]].std())

train_set = train_set.copy()
test_set = test_set.copy()

for i in range(0, len(varName), 1):
    train_set.loc[:, varName[i]] = (train_set[varName[i]].copy() - mean_li[i] + 1.0) / std_li[i]
    test_set.loc[:, varName[i]] = (test_set[varName[i]].copy() - mean_li[i] + 1.0) / std_li[i]



In [3]:
def compute_distances(P, C):
    A = (P**2).sum(axis=1, keepdims=True)
 
    B = (C**2).sum(axis=1, keepdims=True).T
 
    return np.sqrt(A + B - 2* np.dot(P, C.T))

In [4]:
test_set

,date,lon,lat,fCO2,Chl,Temp,Salt,month
132516,2010/7/16,173.25,55.25,0.379430,1.281975,-1.738287,0.386586,7
132520,2010/7/16,-168.75,66.75,-2.759659,2.232367,-1.903447,0.381619,7
132524,2010/7/16,173.75,56.75,0.305969,1.245255,-1.702248,0.349873,7
132528,2010/7/16,174.75,56.75,-0.467146,1.222401,-1.791050,0.301272,7
132532,2010/7/16,174.75,57.25,-0.404352,1.380749,-1.741096,0.270506,7
...,...,...,...,...,...,...,...,...
263880,2020/7/16,137.75,13.25,0.921513,0.937207,1.387684,-0.164779,7
263884,2020/7/16,137.25,12.75,0.869110,0.934032,1.396153,-0.232795,7
263888,2020/7/16,137.75,8.75,0.640634,0.961555,1.416420,-0.236239,7
263892,2020/7/16,137.25,14.25,0.956775,0.937405,1.387597,-0.284868,7


In [4]:
def process_df(my_set, varName):
    temp_df = pd.DataFrame()


    dataset = my_set.reset_index(drop=True)
    ycor = dataset.lat
    #ycor = dataset.lon


    temp_df['label'] = dataset[varName[0]]

    temp_df['beta'] = np.ones(dataset.shape[0])

    temp_df[varName[1:4]] = dataset[varName[1:4]]

    alist = dataset.lon
    temp = []
    for i in alist:
        if i < 0:
            i = i+360
        temp.append(i)
    xcor = temp

    cor_df = pd.DataFrame()
    cor_df['xcor'] = xcor
    cor_df['ycor'] = ycor

    a = [[110.0, 0.0], [290.0,0.0], [110.0, 70.0], [290.0, 70.0]]
    b = np.array(a)

    cor_li = cor_df.to_numpy()
    dis_li = compute_distances(cor_li, b)
    dis_df = pd.DataFrame(dis_li)
    temp_df = temp_df.join(dis_df)

    return temp_df



train_data = MYDataset(process_df(my_set=train_set, varName=varName))
test_data = MYDataset(process_df(my_set=test_set, varName=varName))
train_loader = DataLoader(train_data, batch_size=50, shuffle=True, num_workers=0, drop_last=True)
test_loader = DataLoader(test_data, batch_size=50, shuffle=False, num_workers=0)

In [8]:
process_df(my_set=train_set, varName=varName)

,label,beta,Chl,Temp,Salt,0,1,2,3
0,1.124828,1.0,1.105434,0.909855,3.883645,141.510159,45.990488,147.682514,62.450981
1,0.464580,1.0,1.028047,0.851853,3.836275,140.029015,47.308826,146.263888,63.428109
2,1.470986,1.0,1.038277,0.764635,3.788166,140.684487,47.351082,146.414224,62.346812
3,1.471643,1.0,1.294102,0.569216,3.590262,139.868242,48.714731,145.148631,62.274594
4,1.439575,1.0,1.060960,0.465178,3.570974,138.971670,49.830964,144.042095,62.594928
...,...,...,...,...,...,...,...,...,...
5278,0.911355,1.0,0.917004,1.377116,-0.163213,30.751016,152.825472,63.171394,162.482691
5279,0.859159,1.0,0.913908,1.385612,-0.231479,30.085295,153.281196,63.404456,163.126102
5280,0.631583,1.0,0.940740,1.405943,-0.234936,29.096821,152.501230,67.243029,164.108577
5281,0.946478,1.0,0.917196,1.377029,-0.283744,30.751016,153.413249,62.053404,162.605735


In [12]:


model = GNNWR(outsize=4)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [6]:


r2 = 0
weightlist = []
for i in range(1,2):
    temp = []
    temp.append(-0.172075)
    temp.append(-0.175203)
    temp.append(0.294790)
    temp.append(0.385374)
    weightlist.append(temp)
out = nn.Linear(4, 1, bias = False)
out.weight = nn.Parameter(torch.tensor(weightlist), requires_grad=False)

def train(epoch):
    model.train()
    train_loss = 0
    global r2
    global out
    for data, coef, label in train_loader:
        data = data.view(data.shape[0], -1)
        label = label.view(data.shape[0], -1)
        optimizer.zero_grad()

        output = model(data)
        output = output.mul(coef)
        output = out(output)

        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        a = output.view(-1).detach().numpy()
        b = label.view(-1).numpy()
        if epoch % 100 == 0:
            r2 = r2_score(a, b)

        train_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(train_loader.dataset)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

def val(epoch):
    model.eval()
    global out
    global r2
    val_loss = 0

    label_li = np.array([])
    out_li = np.array([])

    with torch.no_grad():
        for data, coef, label in test_loader:
            data = data.view(data.shape[0], -1)
            label = label.view(data.shape[0], -1)

            output = model(data)
            output = output.mul(coef)
            output = out(output)

            loss = criterion(output, label)

            a = output.view(-1).detach().numpy()
            b = label.view(-1).numpy()
            out_li = np.append(out_li, a)
            label_li = np.append(label_li, b)
            

            val_loss += loss.item()*data.size(0)
        val_loss = val_loss/len(test_loader.dataset)
        label_li = np.array(label_li).reshape(-1)
        out_li = np.array(out_li).reshape(-1)
        if epoch % 100 == 0:
            r2 = r2_score(out_li, label_li)
        #print(out_li)
        print('Epoch: {} \tTraining Loss: {:.6f} \tR2: {:.6f}'.format(epoch, val_loss, r2))
        

In [13]:
for epoch in tqdm(range(1, 1000+1)):
    train(epoch)
    val(epoch)
    if epoch % 2 ==0:
         clear()


  5%|▌         | 51/1000 [00:35<10:57,  1.44it/s]

Epoch: 51 	Training Loss: 0.500519
Epoch: 51 	Training Loss: 0.567071 	R2: 0.000000


  5%|▌         | 51/1000 [00:35<11:04,  1.43it/s]


KeyboardInterrupt: 

In [11]:
for data, coef, label in train_loader:
    print(label.view(50,-1))

tensor([[-0.0715],
        [-3.7681],
        [ 0.7460],
        [-0.5766],
        [ 0.5659],
        [ 1.4982],
        [-0.0624],
        [ 0.2875],
        [-0.1769],
        [ 0.8362],
        [-0.3595],
        [-0.7931],
        [ 0.5001],
        [-0.0829],
        [ 0.1076],
        [ 1.0129],
        [ 0.8379],
        [ 0.1282],
        [ 2.9354],
        [-2.1299],
        [-3.2739],
        [ 0.9454],
        [ 0.1600],
        [-0.4161],
        [-0.0644],
        [ 0.7769],
        [ 0.5043],
        [ 0.2976],
        [ 0.1073],
        [ 1.0696],
        [-0.4767],
        [-1.1549],
        [ 1.2759],
        [-1.6232],
        [ 1.5399],
        [-0.5788],
        [-3.5558],
        [ 0.4533],
        [ 0.1302],
        [-0.3603],
        [-1.5907],
        [-0.6582],
        [-2.7911],
        [-0.1711],
        [ 0.3119],
        [ 0.8989],
        [ 0.0480],
        [ 1.7714],
        [ 0.4933],
        [-0.2207]])
tensor([[-0.1579],
        [-1.1951],
        [ 2